# Notebook to demo simople DSP interactives

Run code cell by clicking in cell to select it then click `Run` button in toolbar.

In [ ]:
#RUN THIS CELL FIRST TO IMPORT NECESSARY PACKAGES

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from ipywidgets import interact

## Simple demo of adding waves in continuous domain

Simple exploration of adding two sine waves, where second wave has frequency and and amplitude multiples of, and phase shift from, a base sine wave.

In [ ]:
@interact(freq = (1,5),phase=(0,2*np.pi), freqmult=(0.5,20), amp=(0.0,2.0)
          )
def sine_wave(freq=1, phase=np.pi, freqmult=1, amp=1.0):
    ''' Plot two sine waves added together.
        freq: base frequency of one wave
        phase: phase shift of added sine wave
        freqmult: frequency of added sine wave as multiple of base frequency
        amp: amplitide of added sine wave as multiple of base amplitude
    '''
    
    df = pd.DataFrame({'x':np.arange(10, step=0.01)})
    df['y'] = np.sin(freq * df['x'])
    df['y2'] =  amp*np.sin(freqmult*freq * df['x'] + phase)
    df['y3']=df['y']+df['y2']
    
    ax=df.plot(x='x',y='y', color='lightblue')
    df.plot(x='x',y='y2', color='lightgreen', ax=ax)
    df.plot(x='x',y='y3', color='red', style = '--', ax=ax)
    
    #return df

## Looking at sine waves in discrete time

Simple demo of how sine waves look in discrete time.

Increase number of samples to show effect of increasing sample rate.

In [ ]:
@interact(fn=['sin','cos', 'lin'], samples=(1,50,1))
def fnplot(fn, samples=10):
    ''' Plot simple discretised signal.
        fn: function
        samples: no. of samples per 2 pi'''

    _fn={'sin':np.sin, 'cos':np.cos, 'lin':np.ones}
    x = pd.DataFrame({'s':np.linspace(0.1, 2 * np.pi, samples)})
    x['sin']=np.sin(x['s'])
    x['cos']=np.cos(x['s'])
    x['lin']=np.ones(len(x))
    x.reset_index(inplace=True)
    out=x[fn]
    
    markerline, stemlines, baseline = plt.stem(x['index'], out)
    fig = plt.setp(baseline, color='grey', linewidth=2)


## Simple Moving Average Filter

Apply a simple moving average filter with specified window (number of considered samples) size.

In the case of a simple sine wave, this effectively introduces a phase shift?

In [ ]:
@interact(fn=['sin','cos', 'lin'], samples=(1,50,1), window=(3,7))
def mafnplot(fn, samples=10, window=3):
    ''' Plot moving average over simple discretised signal.
        fn: function
        samples: no. of samples per 2 pi'''
    
    _fn={'sin':np.sin, 'cos':np.cos, 'lin':np.ones}
    x = pd.DataFrame({'s':np.linspace(0.1, 2 * np.pi, samples)})
    x['sin']=np.sin(x['s'])
    x['cos']=np.cos(x['s'])
    x['lin']=np.ones(len(x))
    out=x[fn]
    
    x['ma']=out.rolling(window).mean()
    
    x.reset_index(inplace=True)
    
    
    fig=plt.figure(1)
        
    ax1 = plt.subplot(211)
    markerline, stemlines, baseline = plt.stem(x['index'], out, basefmt='grey')
    plt.title('Original signal'.format(window))
    
    plt.subplot(212)
    plt.stem(x['index'],x['ma'].fillna(0), basefmt='grey')
    plt.title('Moving average (window size {})'.format(window))
    
    #Add space between plots
    fig.subplots_adjust(hspace=1.2)
    #print(x)
    #fig = plt.setp(baseline, color='grey', linewidth=2)

If we have a combination of signals, then we can get a low pass filter effect from the moving average.

For example, set the samples high, `freqmult` high (e.g. 20) and the `amp` low (e.g. 0.1) then change the window size.

In [ ]:
@interact( samples=(1,100,5), window=(3,7),freq = (1,5),phase=(0,2*np.pi), freqmult=(0.5,20), amp=(0.0,2.0))
def dmafnplot(samples=100, window=3,freq=1,phase=0, freqmult=15, amp=0.1):
    ''' Plot moving average over simple discretised signal.
        fn: function
        samples: no. of samples per 2 pi'''

    df = pd.DataFrame({'x':np.linspace(0.1, 2 * np.pi, samples)})
    df['y'] = np.sin(freq * df['x'])
    df['y2'] =  amp*np.sin(freqmult*freq * df['x'] + phase)
    df['y3']=df['y']+df['y2']
    ax=df.plot(x='x',y='y3', color='red')
    df.plot(x='x',y='y', color='blue', ax=ax)
    df.plot(x='x',y='y2', color='green', ax=ax);
    
    df['ma']=df['y3'].rolling(window).mean()
    
    df.reset_index(inplace=True) 
    
    fig=plt.figure(1)
        
    ax1 = plt.subplot(211)
    markerline, stemlines, baseline = plt.stem(df['index'], df['y3'], basefmt='grey')
    plt.title('Mixed signal'.format(window))
    
    plt.subplot(212)
    plt.stem(df['index'],df['ma'].fillna(0), basefmt='grey')
    plt.title('Moving average (window size {})'.format(window))
    
    fig.subplots_adjust(hspace=1.2)
    #print(x)
    #fig = plt.setp(baseline, color='grey', linewidth=2)

#difference  between consecutive points - 2 term difference
3 term average  - rolling mean over N (N odd? 3, 7)